# Data Loading & Processing (Setup)

## Data Loading

### Import libraries

In [15]:
import kagglehub
import os
import shutil

### Download datasets

In [16]:
dest_folder = "../data/raw"
print("Downloading MovieLens 100K from KaggleHub...")
dataset_path = kagglehub.dataset_download("prajitdatta/movielens-100k-dataset")

# Go inside the actual dataset folder (usually /.../ml-100k)
dataset_subfolder = os.path.join(dataset_path, "ml-100k")
if not os.path.exists(dataset_subfolder):
    raise Exception(f"Expected 'ml-100k' folder not found in {dataset_path}")

os.makedirs(dest_folder, exist_ok=True)

# Copy each file inside ml-100k to dest_folder
for filename in os.listdir(dataset_subfolder):
    src_file = os.path.join(dataset_subfolder, filename)
    dst_file = os.path.join(dest_folder, filename)
    shutil.copy2(src_file, dst_file)

print(f"Dataset successfully copied to: {dest_folder}")

Dataset successfully copied to: ../data/raw


## Data Preprocessing

### Setup folders and import

In [17]:
import pandas as pd

# Make sure the processed folder exists
os.makedirs("../data/processed", exist_ok=True)

### Convert u.data - item - user - occupation - genres to CSV files

In [18]:
ratings = pd.read_csv("../data/raw/u.data", sep='\t', header=None,
                      names=["user_id", "item_id", "rating", "timestamp"])
ratings = ratings.drop(columns=["timestamp"])

ratings.to_csv("../data/processed/ratings.csv", index=False)
ratings.head()

movie_columns = [
    "item_id", "title", "release_date", "video_release_date", "IMDb_URL",
    "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

movies = pd.read_csv("../data/raw/u.item", sep='|', encoding='latin-1', header=None, names=movie_columns)
movies.to_csv("../data/processed/movies.csv", index=False)
movies.head()

users = pd.read_csv("../data/raw/u.user", sep='|', header=None,
                    names=["user_id", "age", "gender", "occupation", "zip_code"])
users.to_csv("../data/processed/users.csv", index=False)
users.head()

occupations = pd.read_csv("../data/raw/u.occupation", header=None, names=["occupation"])
occupations.to_csv("../data/processed/occupations.csv", index=False)
occupations.head()

genres = pd.read_csv("../data/raw/u.genre", sep='|', header=None, names=["genre", "genre_id"])
genres.to_csv("../data/processed/genres.csv", index=False)
genres.head()

,genre,genre_id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4


## Data Splitting

### Setup folder and libraries

In [19]:
from pathlib import Path
import pandas as pd

# Make sure the curated folder exist
os.makedirs("../data/curated", exist_ok=True)

### Data Split

In [20]:
def split_per_user(df, test_ratio=0.2, min_ratings=5):
    train_rows = []
    test_rows = []

    for user_id, group in df.groupby("user_id"):
        if len(group) < min_ratings:
            # Not enough ratings → all go to train
            train_rows.extend(group.to_dict("records"))
        else:
            test_count = max(1, int(len(group) * test_ratio))
            test_sample = group.sample(test_count, random_state=42)
            train_sample = group.drop(test_sample.index)

            test_rows.extend(test_sample.to_dict("records"))
            train_rows.extend(train_sample.to_dict("records"))

    train_df = pd.DataFrame(train_rows)
    test_df = pd.DataFrame(test_rows)
    return train_df, test_df

# --- Paths ---
processed_dir = Path("../data/processed")
curated_dir = Path("../data/curated")
curated_dir.mkdir(parents=True, exist_ok=True)

# --- Load full dataset ---
ratings = pd.read_csv(processed_dir / "ratings.csv")

# --- Perform smart split ---
train_df, test_df = split_per_user(ratings, test_ratio=0.2, min_ratings=5)

# --- Save ---
train_df.to_csv(curated_dir / "train.csv", index=False)
test_df.to_csv(curated_dir / "test.csv", index=False)

print("Per-user stratified split complete.")

✅ Per-user stratified split complete.
